In [1]:
'''Vector database:'''

'Vector database:'

In [1]:
# https://prnt.sc/J9vD-gL2WwfC
# https://prnt.sc/ngBASrWXpwxh

In [2]:
!pip install pinecone-client

   ---------------------------------------- 0.0/215.5 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/215.5 kB 1.3 MB/s eta 0:00:01
   --------------- ------------------------ 81.9/215.5 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------  215.0/215.5 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 215.5/215.5 kB 1.6 MB/s eta 0:00:00


In [9]:
from pinecone import Pinecone, ServerlessSpec, PodSpec
from dotenv import load_dotenv
import os

In [10]:
load_dotenv()

True

In [11]:
pc = Pinecone(api_key=os.environ['API_KEY_PINECONE'])

In [14]:
pc.create_index(
  name="pod-index",
  dimension=1536,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-west-2"
  )
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-04', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '81e90a76d385a16f9f8a3c9e9f0e2280', 'Date': 'Thu, 16 May 2024 13:06:48 GMT', 'Server': 'Google Frontend', 'Content-Length': '200', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Bad request: Your free plan does not support indexes in the us-west-2 region of aws. To create indexes in this region, upgrade your plan."},"status":400}
